In [1]:
from pymongo import MongoClient
import time 
import multiprocessing

In [2]:

print("Số lượng cpu : ", multiprocessing.cpu_count())

Số lượng cpu :  12


In [3]:
client = MongoClient("mongodb://192.168.40.162:60000")

In [4]:
db = client.hoang_test
collection_name = db["profile_staging"]

In [5]:
collection_name.find_one()

{'_id': 'aiwqkjTEzF2Hr5cG3sEgBg==vuong_thi_lan_anh',
 'create_date': 1626199462,
 'dob': {'final': [],
  'sources': [{'value': [Decimal128('19870909')],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'gender': {'final': [],
  'sources': [{'value': ['F'], 'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'habitual_address': None,
 'id_card': {'final': [],
  'sources': [{'value': [{'id_card_number': 'aiwqkjTEzF2Hr5cG3sEgBg==',
      'id_card_loc': 'Lào Cai'}],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'last_update': 1626199462,
 'names': {'final': [],
  'sources': [{'value': [{'full_name': 'Vương Thị Lan Anh'}],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {},
  'no_accent_name': None},
 'no_accent_name': 'vuong_thi_lan_anh',
 'phone_number': None,
 'resident_address': None,
 'social_insurance_id': None,
 'tax_code': None,
 'indexs': {'resident': None, 'yob': Decimal128('19870909')}}

In [ ]:
items = collection_name.find({"gender.sources.value" : "F" }, 
                            {"_id": 0, 
                             "dob.sources.value": 1,
                             "gender.sources.value": 1}).limit(2)

In [ ]:
for item in items:
    print(item)

{'dob': {'sources': [{'value': [Decimal128('19870909')]}]}, 'gender': {'sources': [{'value': ['F']}]}}
{'dob': {'sources': [{'value': [Decimal128('19791110')]}]}, 'gender': {'sources': [{'value': ['F']}]}}


In [20]:
t1 = time.time()
insert_result = collection_name.update_many({ "indexs": { '$exists': False}},
[{"$set": {"indexs": {"resident": None, "yob": None, "gender": None}}}])
print("Thoi gian chay: ", time.time() - t1)
print(insert_result)

Thoi gian chay:  1095.8782997131348


In [21]:
t1 = time.time()
insert_result = collection_name.count_documents({ "indexs": { '$exists': False}})
print("Thoi gian chay: ", time.time() - t1)
print(insert_result)

Thoi gian chay:  387.53938388824463
0


# Tao truong yob - Nam sinh

In [22]:
# dob not null
t1 = time.time()
counts = collection_name.count_documents(filter={ "dob": { '$ne': None}})
print("Thoi gian chay: ", time.time() - t1)

Thoi gian chay:  145.5190896987915


In [23]:
print(counts)

38053295


In [6]:
collection_name.update_many({}, 
    [{"$set": {"indexs.yob": None}}])

In [7]:
t1 = time.time()
counts = collection_name.update_many({ "dob": { '$ne': None}}, 
    [{"$set" :
        {"indexs.yob": 
            {"$toInt": 
                {"$trunc": 
                    [{"$divide": 
                        [{"$first": { "$first": "$dob.sources.value"}}, 10000]
                    }]
                }
            }
        }
    }]
)
print("Thoi gian chay: ", time.time() - t1)

Thoi gian chay:  4380.383522748947


In [8]:
counts.modified_count

38053295

In [34]:
# dob null & id_card not null
documentsDobNull = collection_name.find({"dob": None, }) 

In [ ]:
reUpdate.modified_count

In [6]:
testDB = client.testDB
temp = testDB['temp']

In [8]:
temp.insert_one({'dob': {'final': [],
  'sources': [{'value': [19870909],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {}}})

In [11]:
temp.find_one()

{'_id': ObjectId('615515d28cf6eed7f5c934c9'),
 'dob': {'final': [],
  'sources': [{'value': [19870909], 'source': ['tax_person_new_2019_std']}],
  'meta_data': {}}}

In [31]:
re = temp.update_many({},
    [{"$set": {'yob1': {'$first': {'$first': "$dob.sources.value"}}}}])

In [32]:
re.modified_count

1

In [33]:
temp.find_one()

{'_id': ObjectId('615515d28cf6eed7f5c934c9'),
 'dob': {'final': [],
  'sources': [{'value': [19870909], 'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'yob': 19870909,
 'yob1': 19870909}

In [ ]:
document_ids = collection_name.find({}, {"_id":1}).distinct('_id') #list of all ids

# Tao truong gender

In [33]:
t1 = time.time()
reUpdateGender = collection_name.update_many({ "gender": { '$ne': None}}, 
    [{"$set" :
        {"indexs.gender": {
             "$cond": { "if": { "$eq": ["$gender.sources.value", "M"] }, "then": 1, "else": 0 }
        }
        }
    }]
)
print("Thoi gian chay: ", (time.time() - t1)/60)

Thoi gian chay:  73.61827255090078


In [ ]:
print(reUpdateGender)

In [10]:
testDB = client.testDB
temp = testDB['temp']

In [29]:
t1 = time.time()
reUpdateGender = temp.update_many({}, 
    [{"$set" :
        {"indexs.gender2": {
            "$cond": { "if": { "$eq": ["$gender.sources.value", "M"] }, "then": 1, "else": 0 }
        }
        }
    }]
)
print("Thoi gian chay: ", (time.time() - t1)/60)

Thoi gian chay:  3.2571951548258465e-05


In [13]:
temp.update_many({}, 
    [{"$set":
    {

         'gender': {
            'sources': [{'value': ['F'], 'source': ['tax_person_new_2019_std']}],
        }
    } 
    }]
)

In [30]:
temp.find_one()

{'_id': ObjectId('615515d28cf6eed7f5c934c9'),
 'dob': {'final': [],
  'sources': [{'value': [19870909], 'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'yob': 19870909,
 'yob1': 19870909,
 'id_card': {'final': [],
  'sources': [{'value': [{'id_card_number': 'aiwqkjTEzF2Hr5cG3sEgBg==',
      'id_card_loc': 'Lào Cai'}],
    'source': ['tax_person_new_2019_std']}]},
 'cmt': 'error! java.lang.Exception: Illegal base64 character 24',
 'indexs': {'yob': 1987,
  'yob1': 1987,
  'gender': 0,
  'gender1': 0,
  'gender2': 0},
 'cmt1': 'aiwqkjTEzF2Hr5cG3sEgBg==',
 'cmt2': 'error! java.lang.Exception: Illegal base64 character 24',
 'test_value': 'just testing .. 1',
 'gender': {'sources': [{'value': ['F'],
    'source': ['tax_person_new_2019_std']}]}}

In [32]:
collection_name.find_one({"dob": {"$ne": None}})

{'_id': 'oinyE6ArLFYS0g9cU24W6w==vu_thi_thu',
 'create_date': 1626199462,
 'dob': {'final': [],
  'sources': [{'value': [Decimal128('19910702')],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'gender': {'final': [],
  'sources': [{'value': ['F'], 'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'habitual_address': None,
 'id_card': {'final': [],
  'sources': [{'value': [{'id_card_number': 'oinyE6ArLFYS0g9cU24W6w==',
      'id_card_loc': 'Namdinh'}],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {}},
 'last_update': 1626199462,
 'names': {'final': [],
  'sources': [{'value': [{'full_name': 'Vũ Thị Thu'}],
    'source': ['tax_person_new_2019_std']}],
  'meta_data': {},
  'no_accent_name': None},
 'no_accent_name': 'vu_thi_thu',
 'phone_number': None,
 'resident_address': None,
 'social_insurance_id': None,
 'tax_code': None,
 'indexs': {'resident': None, 'yob': 1991}}